In [ ]:
import pandas as pd
import numpy as np

import copy

import warnings
from numba.core.errors import NumbaDeprecationWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=NumbaDeprecationWarning)
warnings.filterwarnings('ignore')

In [ ]:
data_train = pd.read_csv('../data/mnist/mnist_train.csv')
data_test = pd.read_csv('../data/mnist/mnist_test.csv')

N_train = 6000
n_train = N_train/len(data_train)
N_test = 2000
n_test = N_test/len(data_test)

data_train = data_train.sample(frac=n_train, random_state=9)
data_test = data_test.sample(frac=n_test, random_state=9)

X_train = data_train.iloc[:, 1:].values
y_train = data_train.iloc[:, 0].values

X_test = data_test.iloc[:, 1:].values
y_test = data_test.iloc[:, 0].values

#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaler.fit(X_train)

#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

def transform_line(line, seuil, taille_contour):
    # [moyenne, ecart-type, nb de blanc, nb de noir, contour, centre]
    l_tot = []
    l_tot.append(line.mean())
    l_tot.append(line.std())
    l_tot.append(np.count_nonzero(line))
    a = 0
    for j in range(len(line)):
        if line[j] > seuil :
            a+=1
    l_tot.append(a)
    m_cont=[]
    m_centre=[]
    for j in range(len(line)):
        if j < taille_contour*28 or j > 783 - taille_contour*28 or j%28 < taille_contour or j%28 > 27 - taille_contour:
            m_cont.append(line[j])
        else :
            m_centre.append(line[j])
    l_tot.append(np.mean(m_cont))
    l_tot.append(np.mean(m_centre))
    return np.round(l_tot,4)

def transfo_perso(df, seuil, taille_contour):
    new_X = pd.DataFrame(columns=['moyenne', 'ecart-type', 'nb de blanc', 'nb de noir', 'contour', 'centre'])
    for i in range(len(df)):
        new_X.loc[i] = transform_line(df[i], seuil, taille_contour)
    return new_X

In [ ]:
new_X_train = transfo_perso(X_train, 200, 4)
new_X_test = transfo_perso(X_test, 200, 4)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(new_X_train)
X_train = scaler.transform(new_X_train)
X_test = scaler.transform(new_X_test)

In [ ]:
import torch

# Set the hyperparameters for data creation
NUM_CLASSES = 10
NUM_FEATURES = 6
RANDOM_SEED = 9


# 2. Turn data into tensors
X_train = torch.from_numpy(X_train).type(torch.float)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
X_test = torch.from_numpy(X_test).type(torch.float)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)
print(X_train[:5], y_train[:5])

In [ ]:
from torch import nn

class Model(nn.Module):
    def __init__(self, input_features, output_features, hidden_units):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=hidden_units, out_features=hidden_units), # how many classes are there?
            nn.ReLU(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=hidden_units, out_features=output_features),
            
        )
    
    def forward(self, x):
        return self.linear_layer_stack(x)

# Create an instance of BlobModel and send it to the target device
model = Model(input_features=NUM_FEATURES, 
                    output_features=NUM_CLASSES, 
                    hidden_units=20)
model

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 
                            lr=0.1)

In [ ]:
# Fit the model
torch.manual_seed(42)

# Set number of epochs
epochs = 5000

for epoch in range(epochs+1):
    ### Training
    model.train()

    # 1. Forward pass
    y_logits = model(X_train) # model outputs raw logits 
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) 
    acc = accuracy_fn(y_true=y_train, y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model(X_test)
        test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
        # 2. Calculate test loss and accuracy
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test,y_pred=test_pred)

    if epoch % (epochs/10) == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%") 

In [ ]:
X = pd.DataFrame(X_test)
y = pd.Series(y_test)
X.columns = ['moyenne', 'ecart-type', 'nb de blanc', 'nb de noir', 'contour', 'centre']

class MonModel():
    def __init__(self, model):
        self.model = model
    def predict(self, X):
        X = torch.from_numpy(X.values).float()
        return pd.DataFrame((self.model(X).detach().numpy())).idxmax(axis=1)
    
my_model = MonModel(model)

In [ ]:
import antakia
explain = antakia.Xplainer(X, y, my_model)
explain.interface()

In [ ]:
l = explain.result()[3]['y']
l = list(l)
for i in range(10):
    print(f'{i} : {l.count(i)}')

In [ ]:
X_1 = explain.result()[3]['X']
y_1 = explain.result()[3]['y']